In [1]:
import os
from scipy import stats
from process import *
from utils import *

/home/hongn/miniconda3/envs/xai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pre-defined

In [2]:
TRIALS = {
    "all": TRIAL_LIST,
    "texture": TRIAL_TEXTURE,
    "complexity": TRIAL_COMPLEXITY,
    "orientation": TRIAL_ORIENTATION,
    "brightness": TRIAL_BRIGHTNESS,
    "contrast": TRIAL_CONTRAST,
    "color": TRIAL_COLOR,
    "depth": TRIAL_DEPTH,
    "face": TRIAL_FACE,
    "movement": TRIAL_MOVEMENT,
    "face_texture": TRIAL_FACE_TEXTURE,
}

these_trials = TRIALS["all"]
smap = "face"

def statistic_mw(x, y):
    return stats.mannwhitneyu(x, y)[0]


def iqr(x):
    q1, q3 = np.percentile(x, [25, 75])
    return q3 - q1


def extract_features(ids, trial, smap):
    if smap is not None:
        smap = trial.load_saliency_map(smap)
        smap_vs = trial.load_saliency_map("red")

    features = []
    for subject in ids:
        sub = Subject(DATA_ROOT, subject)
        out = sub.extract_fixations(trial_name=this_trial.trial_name)
        analyzer = FixationAnalyzer(DATA_ROOT, out)
        feat = analyzer.average_saliency(smap)
        feat_vs = analyzer.average_saliency(smap_vs)
        # feat = analyzer.number_of_fixations()
        # feat = sub.eye_mov_entropy(trial.trial_name, perplexity=True)
        features.append(feat - feat_vs)
        feats[subject].append(feat - feat_vs)
    return features

In [3]:
class ImageTrial:
    def __init__(self, root, trial_name, smap_dir):
        self.root = root
        self.new_res = True if "new_res" in root else False
        self.path = glob.glob(f"/home/hong/cvi-stimuli/*{trial_name}*")[0]
        self.trial_name = trial_name
        self.smap_dir = smap_dir
        self.ids = glob.glob(os.path.join(root, "*.asc"))
        self.ids = [os.path.basename(d)[:-4] for d in self.ids]

    def load_trial_img(self):
        img = cv2.imread(self.path)
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    def load_saliency_map(self, smap_type):
        image_name = self.trial_name.strip(".jpg")
        filename = f"{image_name}_{smap_type}.jpg"
        path = os.path.join(self.smap_dir, image_name, filename)

        if os.path.exists(path):
            smap = plt.imread(path)
        else:
            os.makedirs(os.path.join(self.smap_dir, image_name), exist_ok=True)
            sal = SaliencyMap(smap_type)
            smap = sal.get_smap(self.load_trial_img())
            cv2.imwrite(path, smap)

        if self.new_res:
            smap = np.pad(smap, ((240, 240), (320, 320)), 'constant')
        return smap.T

    def complexity(self):
        img = self.load_trial_img()
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        stats = measure.regionprops(img)
        areas = [l.area for l in stats]
        rp_tot = img.shape[0] * img.shape[1]
        return sum(areas > (rp_tot / 25000))

    def read_subjects(self, names, vel=False):
        data, frac = {}, {}
        for subject in names:
            sub = Subject(subject)
            trial_data, frac = sub.extract_data(self.trial_name, vel)
            data[subject] = trial_data
            frac[subject] = 1 - frac
        return data, frac

    def read_fixations(self, names):
        fixations = {}
        for subject in names:
            sub = Subject(subject)
            this = sub.extract_fixations(self.trial_name)
            fixations[subject] = this
        return fixations

    def extract_traces(self, names, smap):
        traces = {}
        for subject in names:
            sub = Subject(subject)
            this = sub.extract_trace(self.trial_name, smap)
            traces[subject] = this
        return traces

## Process

In [4]:
import json 
# with open('markov_chains_swapped.json', 'rb') as f:
f = open('markov_chains_swapped.json')
markov_chains_swapped = json.load(f)
markov_chains_swapped.keys()

alldata = {}
for k in markov_chains_swapped.keys():
    alldata[k] = list({k: v for k, v in markov_chains_swapped[k].items() if v}.values())
    # Make every string-int to int 
    for j in range(len(alldata[k])):
        alldata[k][j][1::2] = [int(i) for i in alldata[k][j][1::2]]
        alldata[k][j].insert(0, 10)
        alldata[k][j].insert(0, 'init')


In [31]:
import jajapy as ja
from jajapy.base.Set import Set

In [37]:
nb_trials = 5
# ctmc_ctrl_models = {}
ctmc_models = {}

for key in alldata:
    print(key)
    best_model = None
    quality_best = -1024
    for n in range(1,nb_trials+1):
        current_model = ja.CTMC_random(nb_states=5,
                                labelling=['saccade','complex_scene_fixation','background_fixation', 'face_fixation', 'blink'],
                                self_loop=False,
                                random_initial_state=True,
                                min_exit_rate_time=1,
                                max_exit_rate_time=6)
        current_quality = current_model.logLikelihood(Set(alldata[key], [1] * len(alldata[key]), t=5))
        if quality_best < current_quality: #we keep the best model only
                        quality_best = current_quality
                        best_model = current_model

    ctmc_models[key] = ja.BW().fit(Set(alldata[key], [1] * len(alldata[key]), t=4), initial_model=best_model,  stormpy_output=False, nb_states=5)


1003_2
Learning a CTMC...
|████████████████████████████████████████| 0 in 0.1s (0.00/s) 


TypeError: 'NoneType' object is not subscriptable

In [36]:
Set(alldata[key], [1] * len(alldata[key]), t=4)

In [35]:
alldata[key]

[['init',
  10,
  'saccade',
  52,
  'complex_scene_fixation',
  68,
  'face_fixation',
  60,
  'saccade',
  52,
  'face_fixation',
  432,
  'saccade',
  28,
  'face_fixation',
  482],
 ['init',
  10,
  'complex_scene_fixation',
  172,
  'complex_scene_fixation',
  128,
  'complex_scene_fixation',
  86,
  'complex_scene_fixation',
  76],
 ['init',
  10,
  'saccade',
  44,
  'face_fixation',
  330,
  'saccade',
  28,
  'complex_scene_fixation',
  758,
  'saccade',
  28],
 ['init', 10, 'saccade', 48, 'complex_scene_fixation', 456, 'blink', 198],
 ['init',
  10,
  'complex_scene_fixation',
  132,
  'saccade',
  38,
  'complex_scene_fixation',
  166,
  'saccade',
  48,
  'saccade',
  22,
  'background_fixation',
  156,
  'saccade',
  34,
  'face_fixation',
  76,
  'saccade',
  26],
 ['init',
  10,
  'saccade',
  48,
  'background_fixation',
  268,
  'saccade',
  32,
  'face_fixation',
  172,
  'saccade',
  22,
  'background_fixation',
  90,
  'background_fixation',
  218,
  'saccade',
  22